In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
# from models import pretrained_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)

In [ ]:
import sys
import pandas as pd
sys.path.append(r'D:\skin_disease_detection\backend\app')
from services.training_ensemble_model_service import train_and_evaluate_ensemble
from utils.preprocess import get_data_generators
from models.ensemble_model import train_base_models, stack_and_train_ensemble, evaluate_ensemble_model

metadata_path = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv'
metadata = pd.read_csv(metadata_path)

# Assuming get_data_generators() is already available and returns the required data generators
train_generator, val_generator, test_generator, label_encoder = get_data_generators(
    metadata_path=r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv',
    target_size=(224, 224),
    batch_size=32,
    use_subset=True, 
    sample_size=500
    )

    # use_subset=True  # Optional: set to True for a subset of data
    # sample_size=500   # Optional: define a sample size for testing

# Train base models using the generators
resnet_model, efficientnet_model, densenet_model = train_base_models(train_generator, val_generator, metadata)
# Stack the predictions from the base models and train the meta-model
meta_model = stack_and_train_ensemble([resnet_model, efficientnet_model, densenet_model], val_generator)
# Evaluate the ensemble model
evaluate_ensemble_model([resnet_model, efficientnet_model, densenet_model], meta_model, test_generator)